In [26]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cross_validation import train_test_split

In [8]:
df = pd.read_csv('./train.csv')
train, test = train_test_split(df, test_size=0.2)

In [10]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['none'] = 1-train[label_cols].max(axis=1)
train.describe()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
count,7.668000e+04,76680.000000,76680.000000,76680.000000,76680.000000,76680.000000,76680.000000,76680.000000
mean,4.992766e+11,0.097157,0.010029,0.053743,0.003104,0.049922,0.008542,0.897066
std,2.893588e+11,0.296173,0.099641,0.225511,0.055626,0.217785,0.092028,0.303875
min,2.225664e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.464803e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,5.002274e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,7.499717e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999824e+11,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
train['comment_text'].fillna("cbarcelon", inplace=True)
test['comment_text'].fillna("cbarcelon", inplace=True)

/usr/local/lib/python3.5/dist-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [19]:
import re, string
re_tok = re.compile('([' + string.punctuation + '“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [20]:
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize, min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1, smooth_idf=1, sublinear_tf=1)
trn_term_doc = vec.fit_transform(train['comment_text'])
test_term_doc = vec.fit_transform(test['comment_text'])

In [23]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from scipy import sparse
class NbSvmClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, C=1.0, dual=False, n_jobs=1):
        self.C = C
        self.dual = dual
        self.n_jobs = n_jobs

    def predict(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        return self._clf.predict(x.multiply(self._r))

    def predict_proba(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        return self._clf.predict_proba(x.multiply(self._r))

    def fit(self, x, y):
        # Check that X and y have correct shape
        y = y.values
        x, y = check_X_y(x, y, accept_sparse=True)

        def pr(x, y_i, y):
            p = x[y==y_i].sum(0)
            return (p+1) / ((y==y_i).sum()+1)

        self._r = sparse.csr_matrix(np.log(pr(x,1,y) / pr(x,0,y)))
        x_nb = x.multiply(self._r)
        self._clf = LogisticRegression(C=self.C, dual=self.dual, n_jobs=self.n_jobs).fit(x_nb, y)
        return self

In [32]:
model = NbSvmClassifier(C=4, dual=True, n_jobs=-1)
preds = np.zeros((len(train), len(label_cols)))
for i, j in enumerate(label_cols):
    print('fit', j)
    model.fit(trn_term_doc, train[j])
    preds[:,i] = model.predict_proba(trn_term_doc)[:,1]

fit toxic


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


fit severe_toxic
fit obscene
fit threat
fit insult
fit identity_hate


In [37]:
preds[2300]

array([ 0.02696577,  0.00061184,  0.00335804,  0.0003897 ,  0.00632688,
        0.00099587])